# Добавление цен в справочник КИАС из источника RSA и с сайта Avito

## Loading data

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df_avito = pd.read_excel('avito_cars_all.xlsx')

In [6]:
df_avito.head()

,ID,Brand,Model,Year,Price,Power,Region
0,3876779285,AC,378 GT Zagato 6.2 MT,2012.0,250000,442.0,Санкт-Петербург
1,3265686141,AC,Ace 2.9 AT,1992.0,750000,195.0,Кемеровская обл.
2,3734363330,Acura,ILX 2.4 AMT,2015.0,1900000,201.0,Санкт-Петербург
3,3908015824,Acura,MDX 3.5 AT,2014.0,2550000,290.0,Москва
4,3728192767,Acura,RDX 3.5 AT,2014.0,1936000,273.0,Москва


In [7]:
df_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473978 entries, 0 to 473977
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      473978 non-null  int64  
 1   Brand   473978 non-null  object 
 2   Model   473978 non-null  object 
 3   Year    473977 non-null  float64
 4   Price   473978 non-null  int64  
 5   Power   460835 non-null  float64
 6   Region  461255 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 25.3+ MB


## Cleaning data of Avito prices

In [8]:
# Deleting duplicates
df_avito = df_avito.drop_duplicates(subset=['ID'])
df_avito.info()

<class 'pandas.core.frame.DataFrame'>
Index: 472460 entries, 0 to 473977
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      472460 non-null  int64  
 1   Brand   472460 non-null  object 
 2   Model   472460 non-null  object 
 3   Year    472459 non-null  float64
 4   Price   472460 non-null  int64  
 5   Power   459318 non-null  float64
 6   Region  459738 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 28.8+ MB


In [9]:
# Define the pattern to match 'number.number MT', 'number.number AT', or 'number.number CVT'
pattern = r'(\s+\d+\.\d+)?\s+(MT|AT|CVT|AMT)$'

# Use str.replace to remove the matched pattern
df_avito['Model'] = df_avito['Model'].str.replace(pattern, '', regex=True)

In [10]:
# Filter the DataFrame to keep only cars from 2012 and later
df_avito_filtered = df_avito[df_avito['Year'] >= 2012]

In [11]:
df_avito_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307975 entries, 0 to 473974
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      307975 non-null  int64  
 1   Brand   307975 non-null  object 
 2   Model   307975 non-null  object 
 3   Year    307975 non-null  float64
 4   Price   307975 non-null  int64  
 5   Power   294905 non-null  float64
 6   Region  295254 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 18.8+ MB


In [12]:
# Group by 'производитель' and 'модель' and calculate min and max prices
df_avito_agg = df_avito_filtered.groupby(['Brand', 'Model', 'Year'])['Price'].agg(median_price='median', min_price='min', max_price='max').reset_index()
df_avito_agg.head()

,Brand,Model,Year,median_price,min_price,max_price
0,AC,378 GT Zagato,2012.0,250000.0,250000,250000
1,AITO,M5,2022.0,4599999.5,3600000,6490000
2,AITO,M5,2023.0,5270000.0,3990000,6800000
3,AITO,M5 EV,2022.0,120000800.0,120000800,120000800
4,AITO,M5 EV,2023.0,5394591.0,4500000,6090000


In [13]:
# Save 'avito_cars_agg' to an Excel file
df_avito_agg.to_excel('avito_cars_agg.xlsx', index=False)